#### Steps in this notebook
- Clean Cricsheet data, create basic model
- Run basic model for testing

In [150]:
import pandas as pd
import numpy as np
import math

import sklearn

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

In [2]:
cricsheet_df=pd.read_csv('./data/all_matches.csv')

/Users/kishan/Desktop/IPL/.ipl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
###Load input format dataframe

test_df=pd.read_csv('./submission/inputFile.csv')

test_df

##Venue needs to go through standardizer
## innigns is either 1 or 2
## Batting team and bowling team better to go through standardizer, since KXIP has changed names between seasons
## Batsmen list must be minimum 2 and bowlers list must be minimum 2
## Need to standardize batter and bowlers
## Batting and bowling types of batters and bowlers to be inferred from custom module

,venue,innings,batting_team,bowling_team,batsmen,bowlers
0,MA Chidambaram Stadium,1,Mumbai Indians,Kolkata Knight Riders,"Quinton de Kock, Rohit Sharma, Suryakumar Yadav","Harbhajan Singh, Varun Chakravarthy, Shakib Al..."
1,MA Chidambaram Stadium,2,Kolkata Knight Riders,Mumbai Indians,"Harbhajan Singh, Varun Chakravarthy, Shakib Al...","Quinton de Kock, Rohit Sharma, Suryakumar Yadav"


#### Subsetting the given data

- Remove seasons before 2018
- Remove super over innings (innings>=3)
- Remove unnecessary columns

In [4]:
##no use of other wicket type and other player dismissed columns
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_wicket_type'])].shape[0]==0
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_player_dismissed'])].shape[0]==0

In [5]:
cricsheet_df.drop(columns=['other_wicket_type','other_player_dismissed'],axis=1,inplace=True)

In [6]:
cricsheet_df['start_date']=pd.to_datetime(cricsheet_df['start_date'])

In [7]:
##Filter from 2018 April 7th
cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')

<ipython-input-7-ff7ecb1db089>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')


count                   43894
unique                    149
top       2020-10-18 00:00:00
freq                      520
first     2018-01-05 00:00:00
last      2021-11-04 00:00:00
Name: start_date, dtype: object

In [8]:
cdf=cricsheet_df.loc[cricsheet_df['start_date']>='2018'].reset_index(drop=True)

### EDA

In [9]:
##List of all venues in this
set(cdf['venue'])

{'Arun Jaitley Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Eden Gardens',
 'Holkar Cricket Stadium',
 'M.Chinnaswamy Stadium',
 'MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai',
 'Maharashtra Cricket Association Stadium',
 'Punjab Cricket Association IS Bindra Stadium',
 'Rajiv Gandhi International Stadium',
 'Sawai Mansingh Stadium',
 'Sharjah Cricket Stadium',
 'Sheikh Zayed Stadium',
 'Wankhede Stadium',
 'Wankhede Stadium, Mumbai'}

#### There are 231 different IPL players across teams from 2018. 

In [10]:
##List of all unique players in the dataset
ipl_players=cdf.loc[:,['striker','non_striker','bowler']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

len(ipl_players),ipl_players

(231,
 ['V Kohli',
  'PA Patel',
  'MM Ali',
  'AB de Villiers',
  'SO Hetmyer',
  'S Dube',
  'C de Grandhomme',
  'NA Saini',
  'YS Chahal',
  'UT Yadav',
  'Mohammed Siraj',
  'SR Watson',
  'AT Rayudu',
  'SK Raina',
  'KM Jadhav',
  'RA Jadeja',
  'DA Warner',
  'JM Bairstow',
  'V Shankar',
  'YK Pathan',
  'MK Pandey',
  'CA Lynn',
  'N Rana',
  'RV Uthappa',
  'KD Karthik',
  'AD Russell',
  'Shubman Gill',
  'PP Shaw',
  'S Dhawan',
  'SS Iyer',
  'CA Ingram',
  'RR Pant',
  'KMA Paul',
  'AR Patel',
  'R Tewatia',
  'RG Sharma',
  'Q de Kock',
  'SA Yadav',
  'KA Pollard',
  'Yuvraj Singh',
  'HH Pandya',
  'KH Pandya',
  'BCJ Cutting',
  'MJ McClenaghan',
  'Rasikh Salam',
  'KL Rahul',
  'MA Agarwal',
  'CH Gayle',
  'SN Khan',
  'N Pooran',
  'Mandeep Singh',
  'AM Rahane',
  'JC Buttler',
  'SV Samson',
  'SPD Smith',
  'RA Tripathi',
  'BA Stokes',
  'K Gowtham',
  'JC Archer',
  'JD Unadkat',
  'S Gopal',
  'DS Kulkarni',
  'MS Dhoni',
  'DJ Bravo',
  'SP Narine',
  'DA

In [11]:
### Format seasons
print(set(cdf['season']))

cdf['season']=cdf['season'].astype('str')
print(set(cdf['season']))

{2018, 2019, '2021', '2018', '2020/21', '2019'}
{'2021', '2018', '2020/21', '2019'}


In [12]:
##Remove super overs and repeated super over instance from MI KXIP
cdf=cdf.loc[cdf['innings']<=2]

In [13]:
##Which venues need tweaking? For purposes of training
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Sheikh Zayed Stadium                                   5294
Wankhede Stadium                                       3902
Eden Gardens                                           3802
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
Sawai Mansingh Stadium                                 3286
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
Punjab Cricket Association IS Bindra Stadium           2429
MA Chidambaram Stadium                                 2130
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
MA Chidambaram Stadium, Chepauk, Chennai                498
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Wankhede Stadium, Mumbai                                239
Name: venue, dtype: int64

In [14]:
cdf.loc[cdf['venue']=='MA Chidambaram Stadium, Chepauk, Chennai','venue']='MA Chidambaram Stadium'
cdf.loc[cdf['venue']=='Wankhede Stadium, Mumbai','venue']='Wankhede Stadium'

In [15]:
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Sheikh Zayed Stadium                                   5294
Wankhede Stadium                                       4141
Eden Gardens                                           3802
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
Sawai Mansingh Stadium                                 3286
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
MA Chidambaram Stadium                                 2628
Punjab Cricket Association IS Bindra Stadium           2429
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Name: venue, dtype: int64

In [16]:
##List of all unique teams in the dataset
ipl_teams=cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()
ipl_teams
##Change Delhi Daredevils to Delhi Capitals
##Change Kings XI Punjab to Punjab Kings

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Kings XI Punjab',
 'Rajasthan Royals',
 'Delhi Daredevils']

In [17]:
cdf.loc[cdf['batting_team']=='Delhi Daredevils','batting_team']='Delhi Capitals'
cdf.loc[cdf['bowling_team']=='Delhi Daredevils','bowling_team']='Delhi Capitals'
cdf.loc[cdf['batting_team']=='Kings XI Punjab','batting_team']='Punjab Kings'
cdf.loc[cdf['bowling_team']=='Kings XI Punjab','bowling_team']='Punjab Kings'

In [18]:
##Changed teams list

cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals']

In [19]:
##Extras column has captured type of extras
##At test time we will not have ball by ball information, do we need to hold a split of extras in that case??
cdf.drop(columns=['wides','noballs','byes','legbyes','penalty'],inplace=True)

### Derive new features

In [20]:
cdf['total_runs']=cdf['runs_off_bat']+cdf['extras']

In [21]:
##Get over number
cdf['over_number']=cdf['ball'].apply(lambda x:int(math.ceil(x)))

In [22]:
def phase_cat(over_number):
    '''Determine phase'''
    if over_number<=6:
        phase="PowerPlay"
    if (over_number>6) & (over_number<=15):
        phase="Middle overs"
    if over_number>15:
        phase="Death overs"
    return phase

In [23]:
cdf['phase']=cdf['over_number'].apply(phase_cat)

In [24]:
cdf.to_csv('./data/cleaned_cricsheet.csv',index=False)

In [25]:
cdf.to_excel('./data/cleaned_cricsheet.xlsx',index=False)

### What features can we derive at test time


**Assuming that we can add data to lookup in the submission file**

#### Toss related factors
- Derive who is batting 1st and who is batting 2nd (by order of rows)
- Check team wise
- We will know what kind of game it is, if it is a DAY start or NIGHT start, so we can supply that manually into the submission code


#### Venue related factors
- Venue specific stats, avg PP score in last 'n' games batting 1st (through a lookup table)
- Venue specific stats, avg PP score in last 'n' games batting 2nd (through a lookup table)

#### Team related factors

- When team is batting 1st, what is the avg PP score
- When team is batting 2nd, what is the avg PP score


#### Batsman related factors

- Since the input is a set of all batters in the list, the order will matter. Assuming openers are 1st two, we can measure size of this set to determine number of wickets that have fallen

- We will not have split of how many balls each batter faced. Hence we need to be careful not to over use the batsman feature

- Lookup batter at venue in 1st innings

- Lookup batter at venue in 2nd innings

- One proxy could be to determine average batting time of batsman in PP by batting position at venue

#### Bowler related factors

- Again we dont have info on the split of balls bowled. 

- Lookup bowler at venue in 1st innings how many overs he has generally bowled * ER
 
- Lookup bowler at venue in 2nd innings how many overs he has generally bowled * ER at venue

#### Batsman-Bowler matchups

- Infer batting hand of Batsman, bowling type of bowlers

#### Handle unknowns

- Bowlers like Chetan Sakariya, Avesh Khan are bowling rarely or for the first time. Model must be robust enough to handle "Unknowns"


### Begin model training pipeline

In [26]:
game_start_type="Night"

#### Venue related stats

In [91]:
##Venue related sums
venue_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['venue','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()

<ipython-input-91-e63dc6b94306>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  venue_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['venue','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()


In [28]:
venue_df.to_excel('./data/venue_df.xlsx',index=False)

In [103]:
venue_n=3

In [95]:
vdf=venue_df.groupby(['venue','start_date','match_id','innings']).last(venue_n).reset_index().drop(columns=['start_date','match_id'])

vdf.head()

,venue,innings,total_runs,wicket_type
0,Arun Jaitley Stadium,1,62,1
1,Arun Jaitley Stadium,2,79,0
2,Arun Jaitley Stadium,1,51,2
3,Arun Jaitley Stadium,2,48,3
4,Arun Jaitley Stadium,1,57,0


In [129]:
vdf_lastn=vdf.groupby(['venue','innings'])['total_runs'].mean().reset_index()

vdf_lastn.head()

,venue,innings,total_runs
0,Arun Jaitley Stadium,1,46.285714
1,Arun Jaitley Stadium,2,55.071429
2,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,1,47.500000
3,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,2,48.500000
4,Dubai International Cricket Stadium,1,46.884615


#### Team last few scores

In [104]:
##Team related dimensions

team_n=3
team_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['batting_team','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()

tdf=team_df.groupby(['batting_team','start_date','match_id','innings']).last(team_n).reset_index().drop(columns=['start_date','match_id'])

tdf.head()

<ipython-input-104-dec170683e05>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  team_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['batting_team','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()


,batting_team,innings,total_runs,wicket_type
0,Chennai Super Kings,1,57,1
1,Chennai Super Kings,2,53,2
2,Chennai Super Kings,1,69,1
3,Chennai Super Kings,1,27,1
4,Chennai Super Kings,2,55,2


In [135]:
tdf_lastn=tdf.groupby(['batting_team','innings'])['total_runs'].mean().reset_index()

tdf_lastn.head()

,batting_team,innings,total_runs
0,Chennai Super Kings,1,42.529412
1,Chennai Super Kings,2,43.419355
2,Delhi Capitals,1,46.076923
3,Delhi Capitals,2,51.000000
4,Kolkata Knight Riders,1,45.307692


In [130]:
model_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['match_id','innings','venue','batting_team'])['total_runs'].sum().reset_index()

In [131]:
model_df.head()

,match_id,innings,venue,batting_team,total_runs
0,1136561,1,Wankhede Stadium,Mumbai Indians,39
1,1136561,2,Wankhede Stadium,Chennai Super Kings,42
2,1136562,1,Punjab Cricket Association IS Bindra Stadium,Delhi Capitals,45
3,1136562,2,Punjab Cricket Association IS Bindra Stadium,Punjab Kings,73
4,1136563,1,Eden Gardens,Royal Challengers Bangalore,52


In [132]:
model_df=model_df.merge(vdf_lastn,on=['venue','innings'],suffixes=('','_venueavg'))

model_df.tail()

,match_id,innings,venue,batting_team,total_runs,total_runs_venueavg
361,1216521,2,Sharjah Cricket Stadium,Mumbai Indians,52,51.666667
362,1216527,2,Sharjah Cricket Stadium,Rajasthan Royals,69,51.666667
363,1216531,2,Sharjah Cricket Stadium,Punjab Kings,56,51.666667
364,1216538,2,Sharjah Cricket Stadium,Sunrisers Hyderabad,56,51.666667
365,1216540,2,Sharjah Cricket Stadium,Kolkata Knight Riders,43,51.666667


In [136]:
model_df=model_df.merge(tdf_lastn,on=['batting_team','innings'],suffixes=('','_teamavg'))

model_df.tail()

,match_id,innings,venue,batting_team,total_runs,total_runs_venueavg,total_runs_teamavg
361,1136598,1,Holkar Cricket Stadium,Rajasthan Royals,45,50.000000,47.647059
362,1216514,1,Sheikh Zayed Stadium,Rajasthan Royals,38,44.545455,47.647059
363,1216518,1,Dubai International Cricket Stadium,Rajasthan Royals,47,46.884615,47.647059
364,1216522,1,Dubai International Cricket Stadium,Rajasthan Royals,52,46.884615,47.647059
365,1216496,1,Sharjah Cricket Stadium,Rajasthan Royals,54,47.416667,47.647059


In [138]:
##Make innings and Ordinal feature
model_df['innings']=model_df['innings'].astype('str')

In [143]:
# inn_encoder=LabelEncoder()
venue_encoder=LabelEncoder()
team_encoder=LabelEncoder()

In [144]:
model_df['venue_encoded']=venue_encoder.fit_transform(model_df['venue'])

In [147]:
model_df['team_encoded']=team_encoder.fit_transform(model_df['batting_team'])

In [155]:
##Scale values
minmaxscaler=MinMaxScaler()

In [154]:
mdf_x=model_df.loc[:,['innings','total_runs_venueavg','total_runs_teamavg','venue_encoded','team_encoded']]

In [157]:
X=minmaxscaler.fit_transform(mdf_x)

In [163]:
y=np.array(model_df['total_runs'])

In [164]:
y.shape

(366,)

In [165]:
from sklearn.model_selection import train_test_split

In [166]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [168]:
model=LinearRegression()

In [169]:
model.fit(X_train,y_train)

LinearRegression()

In [183]:
pred=np.round(model.predict(X_test),0)

In [184]:
from sklearn.metrics import mean_squared_error

In [185]:
mean_squared_error(y_test,pred)

85.62162162162163

In [186]:
pred

array([42., 50., 46., 46., 47., 42., 46., 48., 45., 48., 49., 48., 52.,
       47., 46., 48., 51., 48., 47., 44., 51., 44., 47., 44., 47., 42.,
       53., 47., 48., 45., 50., 49., 47., 47., 46., 45., 47., 47., 53.,
       54., 45., 53., 49., 47., 49., 48., 51., 48., 51., 49., 48., 49.,
       51., 55., 42., 56., 46., 51., 48., 46., 48., 48., 50., 45., 54.,
       47., 52., 48., 52., 50., 49., 42., 52., 55.])

In [187]:
y_test

array([44, 45, 41, 46, 57, 33, 51, 59, 27, 50, 58, 77, 68, 51, 36, 57, 39,
       51, 45, 33, 62, 24, 36, 35, 39, 48, 63, 31, 48, 38, 43, 40, 51, 48,
       53, 40, 43, 59, 58, 40, 46, 56, 47, 50, 49, 50, 49, 28, 60, 54, 44,
       40, 57, 67, 49, 62, 53, 56, 30, 57, 47, 52, 50, 38, 66, 47, 55, 49,
       60, 60, 47, 26, 51, 46])

### Powerplay specific features for 2018, 2019. Try to use it on 2021

In [29]:
match_df=cdf.loc[cdf['match_id']==1136565]

In [30]:
match_df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wicket_type,player_dismissed,total_runs,over_number,phase
8950,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.1,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,0,0,NaN,NaN,0,1,PowerPlay
8951,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.2,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,4,0,NaN,NaN,4,1,PowerPlay
8952,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.3,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,1,0,NaN,NaN,1,1,PowerPlay
8953,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.4,Kolkata Knight Riders,Chennai Super Kings,SP Narine,CA Lynn,DL Chahar,0,1,NaN,NaN,1,1,PowerPlay
8954,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.5,Kolkata Knight Riders,Chennai Super Kings,SP Narine,CA Lynn,DL Chahar,0,0,NaN,NaN,0,1,PowerPlay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.3,Chennai Super Kings,Kolkata Knight Riders,DJ Bravo,RA Jadeja,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs
9192,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.4,Chennai Super Kings,Kolkata Knight Riders,RA Jadeja,DJ Bravo,R Vinay Kumar,0,1,NaN,NaN,1,20,Death overs
9193,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.5,Chennai Super Kings,Kolkata Knight Riders,RA Jadeja,DJ Bravo,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs
9194,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.6,Chennai Super Kings,Kolkata Knight Riders,DJ Bravo,RA Jadeja,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs


In [31]:
pp_score1=int(match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1)]['total_runs'].sum())
pp_score2=int(match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2)]['total_runs'].sum())

In [32]:
pp_batters1=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1),['striker','non_striker']].melt()['value'].drop_duplicates().tolist()
pp_batters1.sort()
pp_batters2=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2),['striker','non_striker']].melt()['value'].drop_duplicates().tolist()
pp_batters2.sort()

In [33]:
pp_batters1, pp_batters2

(['CA Lynn', 'N Rana', 'RV Uthappa', 'SP Narine'],
 ['AT Rayudu', 'SK Raina', 'SR Watson'])

In [34]:
pp_bowlers1=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1),['bowler']].melt()['value'].drop_duplicates().tolist()
pp_bowlers1.sort()


pp_bowlers2=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2),['bowler']].melt()['value'].drop_duplicates().tolist()
pp_bowlers2.sort()

In [35]:
pp_bowlers1,pp_bowlers2

(['DL Chahar', 'Harbhajan Singh', 'Imran Tahir', 'RA Jadeja', 'SR Watson'],
 ['AD Russell', 'PP Chawla', 'R Vinay Kumar', 'TK Curran'])

In [36]:
match_df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wicket_type,player_dismissed,total_runs,over_number,phase
8950,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.1,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,0,0,NaN,NaN,0,1,PowerPlay
8951,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.2,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,4,0,NaN,NaN,4,1,PowerPlay
8952,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.3,Kolkata Knight Riders,Chennai Super Kings,CA Lynn,SP Narine,DL Chahar,1,0,NaN,NaN,1,1,PowerPlay
8953,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.4,Kolkata Knight Riders,Chennai Super Kings,SP Narine,CA Lynn,DL Chahar,0,1,NaN,NaN,1,1,PowerPlay
8954,1136565,2018,2018-10-04,MA Chidambaram Stadium,1,0.5,Kolkata Knight Riders,Chennai Super Kings,SP Narine,CA Lynn,DL Chahar,0,0,NaN,NaN,0,1,PowerPlay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.3,Chennai Super Kings,Kolkata Knight Riders,DJ Bravo,RA Jadeja,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs
9192,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.4,Chennai Super Kings,Kolkata Knight Riders,RA Jadeja,DJ Bravo,R Vinay Kumar,0,1,NaN,NaN,1,20,Death overs
9193,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.5,Chennai Super Kings,Kolkata Knight Riders,RA Jadeja,DJ Bravo,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs
9194,1136565,2018,2018-10-04,MA Chidambaram Stadium,2,19.6,Chennai Super Kings,Kolkata Knight Riders,DJ Bravo,RA Jadeja,R Vinay Kumar,1,0,NaN,NaN,1,20,Death overs
